In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<a id='top'></a>
## Index

[Bot](#bot)



## Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from __future__ import division

## Import Datas for dummy Baselines

In [4]:
datas = pd.read_csv('../Data/sample_submission.csv')

In [6]:
A = ['A' for i in datas.index]
B = ['B' for i in datas.index]
C = ['C' for i in datas.index]
D = ['D' for i in datas.index]

In [7]:
datas.correctAnswer=A
datas.to_csv('../Results/submission_all_A.csv',index=False)
datas.correctAnswer=B
datas.to_csv('../Results/submission_all_B.csv',index=False)
datas.correctAnswer=C
datas.to_csv('../Results/submission_all_C.csv',index=False)
datas.correctAnswer=D
datas.to_csv('../Results/submission_all_D.csv',index=False)


## Import Datas & explo

In [8]:
train = pd.read_csv('../Data/training_set.tsv',sep='\t')
valid = pd.read_csv('../Data/validation_set.tsv',sep='\t')
answers=['answerA','answerB','answerC','answerD']

In [9]:
train.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD
0,100001,"When athletes begin to exercise, their heart r...",C,at the tissue level,at the organ level,at the system level,at the cellular level
1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil
2,100003,"When two nuclei are combined into one nucleus,...",D,conversion,reaction,fission,fusion
3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads
4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...


In [11]:
answers=['answerA','answerB','answerC','answerD']
def print_q_a(df,index):
    print df.ix[index].question
    print df.ix[index][answers]

In [12]:
print_q_a(train,2)

When two nuclei are combined into one nucleus, there is a slight change in mass and the release of a large amount of energy. What is this process called?
answerA    conversion
answerB      reaction
answerC       fission
answerD        fusion
Name: 2, dtype: object


## using Word2Vec on training datas

In [13]:
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer

### preparing the sentences and playing with Word2Vec

In [14]:
tokenizer=RegexpTokenizer(r'\w+')
sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index]

In [15]:
sentences

[['when',
  'athletes',
  'begin',
  'to',
  'exercise',
  'their',
  'heart',
  'rates',
  'and',
  'respiration',
  'rates',
  'increase',
  'at',
  'what',
  'level',
  'of',
  'organization',
  'does',
  'the',
  'human',
  'body',
  'coordinate',
  'these',
  'functions',
  'at',
  'the',
  'system',
  'level'],
 ['which',
  'example',
  'describes',
  'a',
  'learned',
  'behavior',
  'in',
  'a',
  'dog',
  'sitting',
  'on',
  'command'],
 ['when',
  'two',
  'nuclei',
  'are',
  'combined',
  'into',
  'one',
  'nucleus',
  'there',
  'is',
  'a',
  'slight',
  'change',
  'in',
  'mass',
  'and',
  'the',
  'release',
  'of',
  'a',
  'large',
  'amount',
  'of',
  'energy',
  'what',
  'is',
  'this',
  'process',
  'called',
  'fusion'],
 ['which',
  'is',
  'a',
  'distinction',
  'between',
  'an',
  'epidemic',
  'and',
  'a',
  'pandemic',
  'the',
  'geographical',
  'area',
  'affected'],
 ['in',
  'which',
  'way',
  'is',
  'the',
  'orbit',
  'of',
  'a',
  'comet'

In [16]:
model = Word2Vec(sentences)

/home/louis/anaconda/lib/python2.7/site-packages/gensim/models/word2vec.py:665: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [19]:
model.most_similar('at',topn=5)

[('measure', 0.9661522507667542),
 ('car', 0.9654197692871094),
 ('fact', 0.9631081819534302),
 ('stream', 0.9626766443252563),
 ('planet', 0.9615350365638733)]

In [20]:
model.most_similar('comet',topn=5)

[('gives', 0.8969342708587646),
 ('demonstrates', 0.8965225219726562),
 ('scale', 0.8928073644638062),
 ('asteroid', 0.8912460803985596),
 ('hurricane', 0.8907577395439148)]

In [21]:
model.most_similar(positive=['orbit', 'comet'], negative=['car'], topn=1)

[('source', 0.838170051574707)]

In [22]:
model['orbit']+model['comet']

array([ 0.0258908 , -0.0140631 ,  0.01379303, -0.00173259,  0.02329137,
        0.00884423, -0.00050931, -0.04525556,  0.00389807,  0.01119427,
       -0.04051065,  0.01347538, -0.00568764,  0.02271687, -0.005059  ,
        0.01504583, -0.02965485,  0.00974148, -0.00085262, -0.02779544,
       -0.01623358,  0.00305359,  0.01644816,  0.02482403, -0.04711986,
       -0.00970684, -0.00809908, -0.02584333, -0.01200304, -0.02060319,
        0.0074841 ,  0.0425647 , -0.01547288, -0.0187436 ,  0.01996035,
        0.0074924 , -0.00485231,  0.01566946,  0.01904504, -0.00105405,
        0.01676118, -0.01857455,  0.02531887, -0.00847057, -0.03489564,
        0.00293606,  0.02567497, -0.01799328,  0.0107691 , -0.00554384,
       -0.00167953,  0.02023991,  0.01404449, -0.01045252, -0.00777429,
       -0.02325445,  0.00659946, -0.00935888,  0.00443046,  0.00643821,
       -0.01316655,  0.01408609,  0.01054836,  0.00173084, -0.00260429,
       -0.02182997,  0.00442865,  0.00033047,  0.02291266, -0.01

## leave one out cross val on trainset

In [23]:
choices=['A','B','C','D']
model_answers=[]
for ind in train.index:
    sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index if i!=ind]
    model = Word2Vec(sentences)
    question = tokenizer.tokenize(train.ix[ind].question.lower())
    answers = [tokenizer.tokenize(train.ix[ind]['answer'+answer].lower())for answer in choices]
    test = []

    list_question_vects=[]
    for w in question:
        if w in model:
            vect = model[w]
            list_question_vects.append(vect)
    question_vector = sum(list_question_vects,0)

    for a in answers:
        list_ansvect=[]
        for w in a:
            if w in model:
                vect = model[w]
                list_ansvect.append(vect)
        sum_ans_vect = sum(list_ansvect,0)
        test.append(np.dot(question_vector,sum_ans_vect)/np.linalg.norm(sum_ans_vect))
    try:
        print choices[argmax(test)]
        model_answers.append(choices[argmax(test)])
    except:
        # si ça merde on met A... très propre comme méthode
        print "la folie"
        model_answers.append('la folie')
    print ind
            
train['first_try']=model_answers
    

C
0
A
1
C
2
D
3
A
4
C
5
la folie
6
A
7
la folie
8
D
9
C
10
A
11
B
12
B
13
la folie
14
la folie
15
C
16
la folie
17
A
18
A
19
la folie
20
la folie
21
A
22
B
23
A
24
A
25
B
26
B
27
B
28
la folie
29
B
30
la folie
31
D
32
D
33
C
34
D
35
D
36
la folie
37
A
38
D
39
A
40
A
41
A
42
D
43
A
44
D
45
D
46
A
47
B
48
A
49
A
50
la folie
51
D
52
B
53
A
54
A
55
C
56
B
57
C
58
la folie
59
D
60
D
61
B
62
C
63
B
64
la folie
65
A
66
la folie
67
A
68
A
69
D
70
C
71
B
72
D
73
C
74
A
75
la folie
76
D
77
B
78
A
79
C
80
A
81
la folie
82
D
83
la folie
84
C
85
D
86
C
87
la folie
88
A
89
D
90
B
91
la folie
92
A
93
A
94
C
95
la folie
96
C
97
B
98
A
99
la folie
100
B
101
A
102
D
103
A
104
B
105
la folie
106
B
107
B
108
C
109
C
110
C
111
B
112
C
113
C
114
A
115
C
116
C
117
D
118
C
119
A
120
D
121
C
122
la folie
123
D
124
D
125
D
126
C
127
B
128
la folie
129
A
130
C
131
la folie
132
A
133
C
134
D
135
A
136
A
137
D
138
C
139
la folie
140
A
141
A
142
D
143
la folie
144
A
145
A
146
D
147
C
148
D
149
B
150
A
151
B
152
la 

KeyboardInterrupt: 

In [ ]:
train['first_try']=model_answers


In [ ]:
sum([1 for i in train.index if train.ix[i].first_try ==train.ix[i].correctAnswer])

In [ ]:
def folie_to_letter(answer,new_letter):
    if answer=='la folie':
        return new_letter
    else:
        return answer
model_answers_A = [folie_to_letter(model_answers[i],'A')for i in range(len(model_answers))]
train['first_try_A']=model_answers_A
print sum([1 for i in train.index if train.ix[i].first_try_A ==train.ix[i].correctAnswer])
print len(train)
print sum([1 for i in train.index if train.ix[i].first_try_A ==train.ix[i].correctAnswer])/len(train)
    

0.24....ça pu, mais on va tester sur le validation set quand même

### creating submission file for validation set

In [ ]:
choices=['A','B','C','D']
model_answers=[]
sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index]
model = Word2Vec(sentences)
for ind in valid.index:

    question = tokenizer.tokenize(valid.ix[ind].question.lower())
    answers = [tokenizer.tokenize(valid.ix[ind]['answer'+answer].lower())for answer in choices]
    test = []

    list_question_vects=[]
    for w in question:
        if w in model:
            vect = model[w]
            list_question_vects.append(vect)
    question_vector = sum(list_question_vects,0)

    for a in answers:
        list_ansvect=[]
        for w in a:
            if w in model:
                vect = model[w]
                list_ansvect.append(vect)
        sum_ans_vect = sum(list_ansvect,0)
        test.append(np.dot(question_vector,sum_ans_vect)/np.linalg.norm(sum_ans_vect))
    try:
        print choices[argmax(test)]
        model_answers.append(choices[argmax(test)])
    except:
        # si ça merde on met A... très propre comme méthode
        print "la folie"
        model_answers.append('A')
    print ind

In [ ]:
            
valid['first_try']=model_answers
datas['correctAnswer']=model_answers
datas.to_csv('../Results/submission_first_try_w2v.csv', index=False)
    

In [ ]:
len(valid)

In [ ]:
submit = pd.DataFrame(copy(valid),columns=valid.keys())

In [ ]:
submit

In [ ]:
datas

In [ ]:
import wikipedia as wp

In [ ]:
wp.search('science')

In [ ]:
wp.page(wp.search('science')[4]).content

# Training on Concepts-CK-12Foundation.txt

In [ ]:
path_to_book='../Ressources/Concepts-CK-12Foundation.txt'

<a id='bot'></a>
[Top](#top)